In [1]:
import sys
sys.path.insert(0,r"C:\Users\LENOVO\Downloads")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
## Lectura de datos     
base_noroccidente = pd.read_csv(r'C:\Users\pc\Downloads\BASE_NOROCCIDENTE.txt', sep=r'¡', engine='python')
base_noroccidente.head()

## Eliminación de variables: 
base_noroccidente2 = base_noroccidente[['CUCONUSE', 'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO','PROM_SUSC', 'CANAL_ENVIO'
       'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

base_noroccidente3 = base_noroccidente2.dropna()

In [3]:
## Group by por nit para calificacion
b4 = base_noroccidente3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_noroccidente4 = pd.merge(base_noroccidente3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_noroccidente4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_noroccidente4['y'] = pd.cut(x=base_noroccidente4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])

In [ ]:
dataTypeSeries = base_noroccidente4.dtypes
dataTypeSeries

cat = base_noroccidente4.select_dtypes(include=['object','category'])
cuanti = base_noroccidente4.select_dtypes(include=['int64','float64'])
  

le = LabelEncoder()
cat = cat.apply(lambda col: le.fit_transform(col))
cat.head(10)

base_noroccidente5=cuanti.join(cat)
print(base_noroccidente5.dtypes)

base_noroccidente5.to_csv(r'C:\Users\pc\Desktop\base_noroccidente_process.csv')

In [2]:
#DESDE ACA 
base_noroccidente5 = pd.read_csv(r'C:\Users\LENOVO\Downloads\base_noroccidente_process.csv', sep=',', engine='python')


X = base_noroccidente5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12','REGIONAL','PROM_SUSC', 'CANAL_ENVIO'     
        # 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO_x', 'CALIFICACION_VECTOR_SERVICIO_y',
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
       'SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO']]

y = base_noroccidente5['y']

In [3]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3, n_jobs = -1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, n_jobs = -1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5, n_jobs = -1),
    GaussianNB()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

instanteFinal = datetime.now()
tiempo = instanteFinal - instanteInicial # Devuelve un objeto timedelta
segundos = tiempo.seconds

KNeighborsClassifier
****Results****
Accuracy: 84.7456%
Log Loss: 2.4533549884879076
DecisionTreeClassifier
****Results****
Accuracy: 82.3421%
Log Loss: 0.4436187184825498
RandomForestClassifier
****Results****
Accuracy: 77.7612%
Log Loss: 0.5264814689475059
MLPClassifier
****Results****
Accuracy: 82.7193%
Log Loss: 0.44003162739807533
LogisticRegression
****Results****
Accuracy: 80.9627%
Log Loss: 0.4920052825093831
GaussianNB
****Results****
Accuracy: 63.1360%
Log Loss: 0.9963122234024159


In [4]:
segundos

14996

In [ ]:
## El mejor modelo de clasificacion fue el KNN

In [5]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
for train_index, test_index in kf.split(X, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    #model = MLPClassifier(alpha=1, max_iter=500).fit(X_train, y_train) 
    model = KNeighborsClassifier(3).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                       y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score   support      pred       AUC
0             0.529378  0.596356  0.560875   44901.0   50582.0  0.829121
1             0.837266  0.861452  0.849187  410017.0  421861.0  0.778976
2             0.457995  0.391189  0.421964  120144.0  102619.0  0.702688
avg / total   0.733988  0.742504  0.737419  575062.0  575062.0  0.867062,              precision    recall  f1-score   support      pred       AUC
0             0.527246  0.537192  0.532173   44902.0   45749.0  0.799766
1             0.838259  0.885880  0.861412  410016.0  433309.0  0.794129
2             0.496703  0.396902  0.441230  120143.0   96003.0  0.724652
avg / total   0.742616  0.756495  0.747919  575061.0  575061.0  0.875460,              precision    recall  f1-score   support      pred       AUC
0             0.509288  0.548941  0.528371   44901.0   48397.0  0.802221
1             0.836382  0.858662  0.847375  410016.0  420938.0  0.774804
2             0.460653  0.405372  0.431248  1201

In [6]:
import pickle

In [8]:
# save

filename='mdl_noroccidente.sav'

pickle.dump(model,open(filename,'wb'))



In [9]:
# load 

mdl_noroccident = pickle.load(open(filename,'rb'))


In [10]:
accuracy_model2 = []
accuracy_model2.append(cr.class_report(y_true = y_test, 
                                       y_pred = mdl_noroccident.predict(X_test),
                                       y_score = mdl_noroccident.predict_proba(X_test)))
 


In [11]:
# Print the accuracy    
print(accuracy_model2)

[             precision    recall  f1-score   support      pred       AUC
0             0.526224  0.580744  0.552142   44901.0   49553.0  0.820394
1             0.837331  0.855998  0.846562  410016.0  419157.0  0.776824
2             0.453404  0.401352  0.425793  120144.0  106351.0  0.704766
avg / total   0.732828  0.739520  0.735665  575061.0  575061.0  0.863334]
